In [120]:
%load_ext autoreload
%autoreload now

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [121]:
from document_segmentation import settings
from document_segmentation.pagexml import GeneraleMissivenDataset

dataset = GeneraleMissivenDataset.from_csv(settings.GENERALE_MISSIVEN_SHEET)

ID in TANAP database                                                                       2047.0
Inv.nr. Nationaal Archief (1.04.02)                                                          1087
Deel v. inventarisnummer                                                                      NaN
Kamer                                                                                   Amsterdam
Beginfolio                                                                                  139.0
Eindfolio                                                                                   139.0
Beginscan                                                                                   299.0
Eindscan                                                                                    299.0
Enkele of dubbele pagina op scan                                                            Enkel
RGP Deel waarin de missive is opgenomen                                                       NaN
RGP pagina waarop de

In [122]:
dataset[5000]

(PageXMLScan(
 	id=NL-HaNA_1.04.02_1142_0204.jpg, 
 	type=['physical_structure_doc', 'text_region', 'pagexml_doc', 'scan'], 
 	stats={"lines": 84, "words": 1163, "text_regions": 13, "columns": 0, "extra": 0, "pages": 0}
 ),
 <Label.IN: 2>)

In [123]:
from document_segmentation.model.page_sequence_tagger import PageSequenceTagger

tagger = PageSequenceTagger()

/Users/carstenschnober/LAHTeR/workspace/document-segmentation/.venv/lib/python3.11/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [124]:
training_dataset = dataset[:100]
test_dataset = dataset[100:200]

In [125]:
tagger.train_(training_dataset, epochs=15, weights=dataset.inverse_frequencies())

4batch [18:51, 282.87s/batch]?, ?epoch/s]
  7%|▋         | 1/15 [18:51<4:24:00, 1131.47s/epoch]

[Loss:	1.317]


4batch [00:00, 19.20batch/s]                    
 13%|█▎        | 2/15 [18:51<1:40:58, 466.02s/epoch] 

[Loss:	1.196]


4batch [00:00, 20.92batch/s]                    
 20%|██        | 3/15 [18:51<50:39, 253.32s/epoch]  

...

[Loss:	0.666]


4batch [00:00, 20.91batch/s]                    
 93%|█████████▎| 14/15 [18:53<00:03,  3.50s/epoch]

[Loss:	0.665]


4batch [00:00, 22.23batch/s]                    
100%|██████████| 15/15 [18:54<00:00, 75.61s/epoch]

[Loss:	0.665]


In [126]:
tagger.precision(test_dataset)

MulticlassPrecision:   0%|          | 0/3.125 [00:00<?, ?batch/s]

In [ ]:
tagger.recall(test_dataset)

MulticlassRecall: 1batch [00:00, 46.68batch/s]                 

[F1 Score:	{'BEGIN': 0.0, 'IN': 0.375, 'END': 1.0}]


{'BEGIN': 0.0, 'IN': 0.375, 'END': 1.0}

In [ ]:
tagger.f1_score(test_dataset)

MulticlassF1Score: 1batch [00:00, 51.29batch/s]                 

[F1 Score:	{'BEGIN': 0.0, 'IN': 0.5454545617103577, 'END': 0.25}]


{'BEGIN': 0.0, 'IN': 0.5454545617103577, 'END': 0.25}

In [ ]:
tagger.accuracy(test_dataset)

In [ ]:
from document_segmentation.pagexml.dataset import Label


preds = tagger(test_dataset)

print(
    "\t".join(
        ("Page ID", "True Label", "Predicted Label", "Correct?", "Predicted Scores")
    )
)
for page_id, true_label, pred, pred_label in zip(
    test_dataset.page_ids(),
    test_dataset.labels(),
    preds,
    preds.argmax(dim=1),
    strict=True,
):
    print(
        "\t".join(
            (
                page_id,
                true_label.name,
                Label(pred_label.item() + 1).name,
                str(Label(pred_label.item() + 1) == true_label),
                str(pred.tolist()),
            )
        )
    )

Page ID	True Label	Predicted Label	Correct?	Predicted Scores
NL-HaNA_1.04.02_1068_0588.jpg	IN	END	False	[0.36294466257095337, 0.20157194137573242, 0.4354833662509918]
NL-HaNA_1.04.02_1068_0589.jpg	IN	END	False	[0.2764382064342499, 0.21182410418987274, 0.5117377042770386]
NL-HaNA_1.04.02_1068_0590.jpg	IN	END	False	[0.21119709312915802, 0.30690351128578186, 0.4818994104862213]
NL-HaNA_1.04.02_1068_0591.jpg	IN	IN	True	[0.021370841190218925, 0.9606594443321228, 0.017969714477658272]
NL-HaNA_1.04.02_1068_0592.jpg	IN	IN	True	[0.23334375023841858, 0.466174453496933, 0.30048173666000366]
NL-HaNA_1.04.02_1068_0593.jpg	IN	END	False	[0.23051902651786804, 0.28804871439933777, 0.48143234848976135]
NL-HaNA_1.04.02_1068_0594.jpg	END	END	True	[0.19421978294849396, 0.2022438645362854, 0.6035364270210266]
NL-HaNA_1.04.02_1068_0595.jpg	BEGIN	END	False	[0.16098953783512115, 0.18695591390132904, 0.6520546078681946]
NL-HaNA_1.04.02_1068_0596.jpg	IN	END	False	[0.13625812530517578, 0.29717302322387695, 0.5665

In [ ]:
from torchview import draw_graph

model_graph = draw_graph(tagger)
print(model_graph.visual_graph)

RuntimeError: Only one of (input_data, input_size) should be specified.